# EPUSP - Random circuits generator - multi datasets - v.12.2022

In [1]:
"""Utility functions for generating random circuits."""

import numpy as np

from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, transpile
from qiskit import QuantumCircuit
from qiskit import Aer

import datetime

from qiskit.circuit.random import random_circuit
from qiskit.providers.fake_provider import FakeManilaV2

from qiskit.circuit import Reset
from qiskit.circuit.library.standard_gates import (IGate, U1Gate, U2Gate, U3Gate, XGate,
                                                   YGate, ZGate, HGate, SGate, SdgGate, TGate,
                                                   TdgGate, RXGate, RYGate, RZGate, CXGate,
                                                   CYGate, CZGate, CHGate, CRZGate, CU1Gate,
                                                   CU3Gate, SwapGate, RZZGate,
                                                   CCXGate, CSwapGate)
from qiskit.circuit.exceptions import CircuitError

In [2]:
def random_circuit(num_qubits, depth, max_operands=3, measure=False, conditional=False, reset=False, seed=None):
    
    if max_operands < 1 or max_operands > 3:
        raise CircuitError("max_operands must be between 1 and 3")

    one_q_ops = [IGate, U1Gate, U2Gate, U3Gate, XGate, YGate, ZGate,
                 HGate, SGate, SdgGate, TGate, TdgGate, RXGate, RYGate, RZGate]
    one_param = [U1Gate, RXGate, RYGate, RZGate, RZZGate, CU1Gate, CRZGate]
    two_param = [U2Gate]
    three_param = [U3Gate, CU3Gate]
    two_q_ops = [CXGate, CYGate, CZGate, CHGate, CRZGate,
                 CU1Gate, CU3Gate, SwapGate, RZZGate]
    three_q_ops = [CCXGate, CSwapGate]

    qr = QuantumRegister(num_qubits, 'q')
    qc = QuantumCircuit(num_qubits)

    if measure or conditional:
        cr = ClassicalRegister(num_qubits, 'c')
        qc.add_register(cr)

    if reset:
        one_q_ops += [Reset]

    if seed is None:
        seed = np.random.randint(0, np.iinfo(np.int32).max)
    rng = np.random.default_rng(seed)

    # apply arbitrary random operations at every depth
    for _ in range(depth):
        # choose either 1, 2, or 3 qubits for the operation
        remaining_qubits = list(range(num_qubits))
        while remaining_qubits:
            max_possible_operands = min(len(remaining_qubits), max_operands)
            num_operands = rng.choice(range(max_possible_operands)) + 1
            rng.shuffle(remaining_qubits)
            operands = remaining_qubits[:num_operands]
            remaining_qubits = [q for q in remaining_qubits if q not in operands]
            if num_operands == 1:
                operation = rng.choice(one_q_ops)
            elif num_operands == 2:
                operation = rng.choice(two_q_ops)
            elif num_operands == 3:
                operation = rng.choice(three_q_ops)
            if operation in one_param:
                num_angles = 1
            elif operation in two_param:
                num_angles = 2
            elif operation in three_param:
                num_angles = 3
            else:
                num_angles = 0
            angles = [rng.uniform(0, 2 * np.pi) for x in range(num_angles)]
            register_operands = [qr[i] for i in operands]
            op = operation(*angles)

            # with some low probability, condition on classical bit values
            if conditional and rng.choice(range(10)) == 0:
                value = rng.integers(0, np.power(2, num_qubits))
                op.condition = (cr, value)

            qc.append(op, register_operands)

    if measure:
        qc.measure(qr, cr)

    return qc

In [15]:
with open('dataset_ideal.dat', 'w') as savefile_ideal, open('dataset_noise.dat', 'w') as savefile_noise, open('dataset_exp.dat', 'w') as savefile_exp:
    
    num_qubits = 3      # num_qubits (int) – number of quantum wires
    num_gates = 3       # depth (int) – layers of operations (i.e. critical path length)
    max_operands = 2    # max_operands (int) – maximum operands of each gate (between 1 and 3)
    num_circuits = 100
        
    # measure (bool) – if True, measure all qubits at the end
    # conditional (bool) – if True, insert middle measurements and conditionals
    # reset (bool) – if True, insert middle resets
    # seed (int) – sets random seed (optional)
  
    for i in range(num_circuits):
        
        # salvando timestamp e resultados.
        ct = datetime.datetime.now()               

        #================================================================================
        # gerando circuito randômico.
        circ = random_circuit(num_qubits, num_gates, max_operands=3, measure=True, conditional=False, reset=False, seed=i)    
        
        # Apresenta o circuito aleatorio gerado.
        print("Circuito aleatório gerado.")
        print(circ)

        #================================================================================
        # criando backend qasm_simulator ideal - sem ruído.
        ideal_backend = Aer.get_backend("qasm_simulator")
        
        # transpiling circuito original para o simulador.
        transpiled_circuit = transpile(circ, ideal_backend)   

        # executando circuito transpilado no simulador ideal.
        ideal_result = ideal_backend.run(transpiled_circuit).result()
        ideal_counts = ideal_result.get_counts()
        
        # --------------------------------------------------------------
        #
        # completando os estados que faltam
        import itertools
        all_bitstrings = ("".join(states) for states in itertools.product("01", repeat=num_qubits))
        new_ideal_counts = {key: ideal_counts.get(key, 0) for key in all_bitstrings}        
        #
        # --------------------------------------------------------------
        
        print("QASM_Simulator" +
                       ";current time: "+ str(ct) + 
                       ";i:" + str(i) + 
                       ";num_qubits:" + str(num_qubits) + 
                       ";num_gates:" + str(num_gates) + 
                       ";results:" + str(new_ideal_counts) + "\n")
            
        savefile_ideal.write("QASM_Simulator" +
                       ";current time: "+ str(ct) + 
                       ";i:" + str(i) + 
                       ";num_qubits:" + str(num_qubits) + 
                       ";num_gates:" + str(num_gates) + 
                       ";results:" + str(new_ideal_counts) + "\n")        
        
        #================================================================================
        # criando backend fake com ruído.
        noise_backend = FakeManilaV2()
        
        # transpiling circuito original para o simulador.
        transpiled_circuit = transpile(circ, noise_backend)   
        
        print("Circuito aleatório transpiled_circuit para FakeManilaV2.")
        print(circ)

        # executando circuito transpilado no simulador fake com ruído.
        noise_result = noise_backend.run(transpiled_circuit).result()
        noise_counts = noise_result.get_counts() 
            
        # --------------------------------------------------------------
        #
        # completando os estados que faltam
        import itertools
        all_bitstrings = ("".join(states) for states in itertools.product("01", repeat=num_qubits))
        new_noise_counts = {key: noise_counts.get(key, 0) for key in all_bitstrings}        
        #
        # --------------------------------------------------------------
            
            
        print("FakeManilaV2" +
                       ";current time: "+ str(ct) + 
                       ";i:" + str(i) + 
                       ";num_qubits:" + str(num_qubits) + 
                       ";num_gates:" + str(num_gates) + 
                       ";results:" + str(new_noise_counts) + "\n")
            
        savefile_noise.write("FakeManilaV2" +
                       ";current time: "+ str(ct) + 
                       ";i:" + str(i) + 
                       ";num_qubits:" + str(num_qubits) + 
                       ";num_gates:" + str(num_gates) + 
                       ";results:" + str(new_noise_counts) + "\n")   
        
        #================================================================================
        print("Resultados IDEAL e FAKE consolidados.")
        
        print(str(ct) + ";" + str(new_ideal_counts) + ";" + str(new_noise_counts) + "\n")          
        savefile_exp.write(str(ct) + ";" + str(new_ideal_counts) + ";" + str(new_noise_counts) + "\n")  
      


Circuito aleatório gerado.
                                         ┌─┐      
q_0: ──■───────────────■───────────────X─┤M├──────
     ┌─┴─┐┌────────────┴─────────────┐ │ └╥┘┌─┐   
q_1: ┤ X ├┤ U3(3.4157,5.8752,5.1262) ├─■──╫─┤M├───
     └─┬─┘└───┬──────────────────┬───┘ │  ║ └╥┘┌─┐
q_2: ──■──────┤ U2(5.735,3.8116) ├─────X──╫──╫─┤M├
              └──────────────────┘        ║  ║ └╥┘
c: 3/═════════════════════════════════════╩══╩══╩═
                                          0  1  2 
QASM_Simulator;current time: 2022-10-10 15:07:17.338924;i:0;num_qubits:3;num_gates:3;results:{'000': 520, '001': 0, '010': 0, '011': 0, '100': 504, '101': 0, '110': 0, '111': 0}

Circuito aleatório transpiled_circuit para FakeManilaV2.
                                         ┌─┐      
q_0: ──■───────────────■───────────────X─┤M├──────
     ┌─┴─┐┌────────────┴─────────────┐ │ └╥┘┌─┐   
q_1: ┤ X ├┤ U3(3.4157,5.8752,5.1262) ├─■──╫─┤M├───
     └─┬─┘└───┬──────────────────┬───┘ │  ║ └╥┘┌─┐
q_2: ──■──────┤ U2(5.73

FakeManilaV2;current time: 2022-10-10 15:07:19.892081;i:5;num_qubits:3;num_gates:3;results:{'000': 964, '001': 10, '010': 16, '011': 4, '100': 9, '101': 1, '110': 0, '111': 20}

Resultados IDEAL e FAKE consolidados.
2022-10-10 15:07:19.892081;{'000': 1024, '001': 0, '010': 0, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0};{'000': 964, '001': 10, '010': 16, '011': 4, '100': 9, '101': 1, '110': 0, '111': 20}

Circuito aleatório gerado.
                             ┌───┐    ┌─┐      
q_0: ──■─────────────────■───┤ T ├────┤M├──────
       │  ┌───────────┐  │  ┌┴───┴┐   └╥┘┌─┐   
q_1: ──┼──┤ Rz(2.353) ├──┼──┤ Sdg ├─■──╫─┤M├───
     ┌─┴─┐└───────────┘┌─┴─┐└─────┘ │  ║ └╥┘┌─┐
q_2: ┤ H ├─────────────┤ Y ├────────■──╫──╫─┤M├
     └───┘             └───┘           ║  ║ └╥┘
c: 3/══════════════════════════════════╩══╩══╩═
                                       0  1  2 
QASM_Simulator;current time: 2022-10-10 15:07:20.246888;i:6;num_qubits:3;num_gates:3;results:{'000': 1024, '001': 0, '010': 0, 

Circuito aleatório transpiled_circuit para FakeManilaV2.
                                                       »
q_0: ──────────────■───────────────────────■───────────»
     ┌─────────────┴──────────────┐┌─────┐ │           »
q_1: ┤ U3(0.92945,5.8321,0.44247) ├┤ Tdg ├─┼───────────»
     └───────────┬───┬────────────┘└─────┘ │U1(2.3185) »
q_2: ────────────┤ H ├─────────────────────■───────────»
                 └───┘                                 »
c: 3/══════════════════════════════════════════════════»
                                                       »
«           ┌─────────────┐       ┌─┐      
«q_0: ──────┤ Rz(0.86688) ├───────┤M├──────
«           └───┬─────┬───┘       └╥┘┌─┐   
«q_1: ──────────┤ Tdg ├────────────╫─┤M├───
«     ┌─────────┴─────┴──────────┐ ║ └╥┘┌─┐
«q_2: ┤ U3(3.2194,5.1317,3.4499) ├─╫──╫─┤M├
«     └──────────────────────────┘ ║  ║ └╥┘
«c: 3/═════════════════════════════╩══╩══╩═
«                                  0  1  2 
FakeManilaV2;current time: 2022-10

Circuito aleatório transpiled_circuit para FakeManilaV2.
      ┌───┐ ┌────────────┐   ┌─┐      
q_0: ─┤ H ├─┤ Ry(5.3662) ├─■─┤M├──────
      └─┬─┘ └───┬───┬────┘ │ └╥┘┌─┐   
q_1: ───■───────┤ X ├──────X──╫─┤M├───
     ┌─────┐   ┌┴───┴┐     │  ║ └╥┘┌─┐
q_2: ┤ Tdg ├───┤ Sdg ├─────X──╫──╫─┤M├
     └─────┘   └─────┘        ║  ║ └╥┘
c: 3/═════════════════════════╩══╩══╩═
                              0  1  2 
FakeManilaV2;current time: 2022-10-10 15:07:24.586444;i:16;num_qubits:3;num_gates:3;results:{'000': 52, '001': 12, '010': 720, '011': 11, '100': 39, '101': 165, '110': 20, '111': 5}

Resultados IDEAL e FAKE consolidados.
2022-10-10 15:07:24.586444;{'000': 0, '001': 0, '010': 826, '011': 0, '100': 0, '101': 198, '110': 0, '111': 0};{'000': 52, '001': 12, '010': 720, '011': 11, '100': 39, '101': 165, '110': 20, '111': 5}

Circuito aleatório gerado.
          ┌───┐┌────────────┐┌─┐      
q_0: ──■──┤ H ├┤ Ry(3.8412) ├┤M├──────
     ┌─┴─┐└─┬─┘└───┬───┬────┘└╥┘┌─┐   
q_1: ┤ X ├──■──────┤ I ├

FakeManilaV2;current time: 2022-10-10 15:07:26.829429;i:21;num_qubits:3;num_gates:3;results:{'000': 757, '001': 7, '010': 245, '011': 3, '100': 11, '101': 0, '110': 1, '111': 0}

Resultados IDEAL e FAKE consolidados.
2022-10-10 15:07:26.829429;{'000': 751, '001': 0, '010': 273, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0};{'000': 757, '001': 7, '010': 245, '011': 3, '100': 11, '101': 0, '110': 1, '111': 0}

Circuito aleatório gerado.
     ┌───┐       ┌───┐     ┌─┐      
q_0: ┤ X ├─X─────┤ I ├─────┤M├──────
     └─┬─┘ │     ├───┤     └╥┘┌─┐   
q_1: ──■───■─────┤ I ├──────╫─┤M├───
       │   │ ┌───┴───┴────┐ ║ └╥┘┌─┐
q_2: ──■───X─┤ Rz(5.3506) ├─╫──╫─┤M├
             └────────────┘ ║  ║ └╥┘
c: 3/═══════════════════════╩══╩══╩═
                            0  1  2 
QASM_Simulator;current time: 2022-10-10 15:07:27.264531;i:22;num_qubits:3;num_gates:3;results:{'000': 1024, '001': 0, '010': 0, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0}

Circuito aleatório transpiled_circuit para Fa

...

Circuito aleatório transpiled_circuit para FakeManilaV2.
                                     ┌───────────────────────────┐┌─┐      
q_0: ─■───────────────────────────■──┤ U3(2.1978,1.1897,0.35632) ├┤M├──────
      │           ┌────────────┐┌─┴─┐└───────────────────────────┘└╥┘┌─┐   
q_1: ─┼───────────┤ Rz(0.4908) ├┤ X ├──────────────X───────────────╫─┤M├───
      │ZZ(4.2876) └────────────┘└─┬─┘              │               ║ └╥┘┌─┐
q_2: ─■───────────────────────────■────────────────X───────────────╫──╫─┤M├
                                                                   ║  ║ └╥┘
c: 3/══════════════════════════════════════════════════════════════╩══╩══╩═
                                                                   0  1  2 
FakeManilaV2;current time: 2022-10-10 15:07:49.177544;i:76;num_qubits:3;num_gates:3;results:{'000': 218, '001': 726, '010': 1, '011': 11, '100': 19, '101': 44, '110': 1, '111': 4}

Resultados IDEAL e FAKE consolidados.
2022-10-10 15:07:49.177544;{'000': 185, 

Circuito aleatório transpiled_circuit para FakeManilaV2.
              ┌─┐      
q_0: ─X──X──X─┤M├──────
      │  │  │ └╥┘┌─┐   
q_1: ─■──X──■──╫─┤M├───
      │  │  │  ║ └╥┘┌─┐
q_2: ─X──■──X──╫──╫─┤M├
               ║  ║ └╥┘
c: 3/══════════╩══╩══╩═
               0  1  2 
FakeManilaV2;current time: 2022-10-10 15:07:51.708464;i:82;num_qubits:3;num_gates:3;results:{'000': 873, '001': 18, '010': 20, '011': 25, '100': 21, '101': 37, '110': 19, '111': 11}

Resultados IDEAL e FAKE consolidados.
2022-10-10 15:07:51.708464;{'000': 1024, '001': 0, '010': 0, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0};{'000': 873, '001': 18, '010': 20, '011': 25, '100': 21, '101': 37, '110': 19, '111': 11}

Circuito aleatório gerado.
        ┌───┐           ┌───┐            ┌─┐      
q_0: ─X─┤ Z ├───────────┤ X ├────────────┤M├──────
      │ ├───┤┌──────────┴───┴───────────┐└╥┘┌─┐   
q_1: ─X─┤ H ├┤ U3(3.7436,3.0795,3.5481) ├─╫─┤M├───
      │ └─┬─┘└──────────┬───┬───────────┘ ║ └╥┘┌─┐
q_2: ─■───■───────────

                ┌───┐                      ┌───┐                 ┌─┐      
q_0: ───────────┤ S ├──────────────────────┤ X ├──────────────X──┤M├──────
                └───┘                      ├───┤              │  └╥┘┌─┐   
q_1: ─────────────■────────────────────────┤ T ├──────────────X───╫─┤M├───
     ┌────────────┴────────────┐┌──────────┴───┴───────────┐┌───┐ ║ └╥┘┌─┐
q_2: ┤ U3(4.326,4.3283,4.1622) ├┤ U3(0.38067,2.3048,4.252) ├┤ T ├─╫──╫─┤M├
     └─────────────────────────┘└──────────────────────────┘└───┘ ║  ║ └╥┘
c: 3/═════════════════════════════════════════════════════════════╩══╩══╩═
                                                                  0  1  2 
FakeManilaV2;current time: 2022-10-10 15:07:54.106859;i:88;num_qubits:3;num_gates:3;results:{'000': 36, '001': 9, '010': 850, '011': 61, '100': 1, '101': 1, '110': 59, '111': 7}

Resultados IDEAL e FAKE consolidados.
2022-10-10 15:07:54.106859;{'000': 0, '001': 0, '010': 989, '011': 0, '100': 0, '101': 0, '110': 35, '111': 

FakeManilaV2;current time: 2022-10-10 15:07:56.045798;i:93;num_qubits:3;num_gates:3;results:{'000': 948, '001': 11, '010': 14, '011': 0, '100': 22, '101': 1, '110': 26, '111': 2}

Resultados IDEAL e FAKE consolidados.
2022-10-10 15:07:56.045798;{'000': 1024, '001': 0, '010': 0, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0};{'000': 948, '001': 11, '010': 14, '011': 0, '100': 22, '101': 1, '110': 26, '111': 2}

Circuito aleatório gerado.
     ┌───┐┌─────────────────────────┐     ┌─┐      
q_0: ┤ X ├┤ U3(3.5454,5.572,1.9199) ├──■──┤M├──────
     ├───┤└─────────┬─────┬─────────┘┌─┴─┐└╥┘┌─┐   
q_1: ┤ X ├──────────┤ Sdg ├──────────┤ X ├─╫─┤M├───
     └─┬─┘          └┬───┬┘          └─┬─┘ ║ └╥┘┌─┐
q_2: ──■─────────────┤ H ├─────────────■───╫──╫─┤M├
                     └───┘                 ║  ║ └╥┘
c: 3/══════════════════════════════════════╩══╩══╩═
                                           0  1  2 
QASM_Simulator;current time: 2022-10-10 15:07:56.389842;i:94;num_qubits:3;num_gates:3;re

Circuito aleatório transpiled_circuit para FakeManilaV2.
            ┌───┐     ┌───┐┌─┐      
q_0: ─■─────┤ X ├─────┤ X ├┤M├──────
      │    ┌┴───┴┐    └─┬─┘└╥┘┌─┐   
q_1: ─X────┤ Sdg ├──────■───╫─┤M├───
      │ ┌──┴─────┴───┐  │   ║ └╥┘┌─┐
q_2: ─X─┤ Rz(3.8636) ├──■───╫──╫─┤M├
        └────────────┘      ║  ║ └╥┘
c: 3/═══════════════════════╩══╩══╩═
                            0  1  2 
FakeManilaV2;current time: 2022-10-10 15:07:58.640783;i:99;num_qubits:3;num_gates:3;results:{'000': 93, '001': 856, '010': 7, '011': 10, '100': 7, '101': 14, '110': 36, '111': 1}

Resultados IDEAL e FAKE consolidados.
2022-10-10 15:07:58.640783;{'000': 0, '001': 1024, '010': 0, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0};{'000': 93, '001': 856, '010': 7, '011': 10, '100': 7, '101': 14, '110': 36, '111': 1}



In [7]:
savefile_ideal.close()
savefile_noise.close()
savefile_exp.close()